In [ ]:
from gevent.pool import Pool
from gevent import monkey
import time
import logging
import sys
import linecache
import random
import undetected_chromedriver as uc
from tkinter import *
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
import requests

logging.disable(logging.CRITICAL)

class ArbFinder(object):
    def __init__(self, URL):
        try:
            self.URL = URL
            self.driver = uc.Chrome()
            self.driver.implicitly_wait(5)
            self.driver.get(URL)
        except Exception as e:
            print("Error initializing browser:", str(e))
        self.sport = 'Football'

    def set_type(self, ASK=0, BID=1):
        try:
            if ASK:
                self.type = "ASK"
                self.driver.find_element(By.XPATH, "//*[@id='KambiBC-content']/div/div/div/div[1]/div[1]/div/div/div/ul/li[1]/div").click()
            elif BID:
                self.type = "BID"
                self.driver.find_element(By.XPATH, "//*[@id='KambiBC-container']/aside/div/nav/div/section[2]/div/ul/li[2]/a/div/span/span[3]").click()
        except Exception as e:
            print("Error setting type:", str(e))

class PS3838API:
    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.base_url = "https://api.ps3838.com/v1"
        self.session = requests.Session()
        self.login()

    def login(self):
        login_url = f"{self.base_url}/user/session"
        headers = {"Content-Type": "application/json"}
        data = {"username": self.username, "password": self.password}
        response = self.session.post(login_url, headers=headers, json=data)
        if response.status_code == 200:
            print("PS3838 login successful")
        else:
            print("PS3838 login failed")

    def get_live_odds(self, sport):
        odds_url = f"{self.base_url}/odds?sport={sport}"
        response = self.session.get(odds_url)
        if response.status_code == 200:
            return response.json()
        else:
            print("Error fetching PS3838 live odds:", response.status_code)
            return None

    def place_bet(self, data):
        bet_url = f"{self.base_url}/bets"
        headers = {"Content-Type": "application/json"}
        response = self.session.post(bet_url, headers=headers, json=data)
        if response.status_code == 200:
            return response.json()
        else:
            print("Error placing bet on PS3838:", response.status_code)
            return None

class App(object):
    def __init__(self):
        self.root = None
        self.ask = None
        self.ps3838_api = None
        self.old_list = ''
        self.num_worker_threads = 2
        self.pool = Pool(self.num_worker_threads)
        self.main_bet_amount = 100  # In SEK
        self.lower_limit = 0.020  # 2% Arbitrage threshold
        self.upper_limit = 0.070
        self.bet_limit = 0.10  # 10% of total funds for each bet
        self.odds_limit = 750
        self.running = False
        self.ignored_exceptions = (NoSuchElementException, StaleElementReferenceException,)
        
        # SpeedyBet API URL
        self.speedybet_api_url = "https://eu-offering-api.kambicdn.com/offering/v2018/speedybetse/event/live/open.json?lang=en_GB&market=SE&client_id=2&channel_id=1&ncid=1705399533648"
        
        # PS3838 API credentials
        self.ps3838_username = "BIA0003Q51"
        self.ps3838_password = "N1ckolas@"

    def PrintException(self):
        exc_type, exc_obj, tb = sys.exc_info()
        f = tb.tb_frame
        lineno = tb.tb_lineno
        filename = f.f_code.co_filename
        linecache.checkcache(filename)
        line = linecache.getline(filename, lineno, f.f_globals)
        print('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))

    def start(self):
        self.running = True

    def stop(self):
        self.running = False

    def get_speedybet_odds(self):
        try:
            response = requests.get(self.speedybet_api_url)
            if response.status_code == 200:
                return response.json()
            else:
                print("Error fetching SpeedyBet odds:", response.status_code)
                return None
        except Exception as e:
            print("Exception in fetching SpeedyBet odds:", str(e))
            return None

    def trading(self):
        if self.running:
            try:
                # Get SpeedyBet odds
                speedybet_odds_data = self.get_speedybet_odds()
                if speedybet_odds_data:
                    # Process SpeedyBet odds data and find arbitrages
                    pass
                    
                # Get PS3838 odds
                if self.ps3838_api:
                    ps3838_odds_data = self.ps3838_api.get_live_odds("Football")
                    if ps3838_odds_data:
                        # Process PS3838 odds data and find arbitrages
                        pass
            except Exception as e:
                self.PrintException()

        # Schedule the trading method to run again
        self.root.after(10, self.trading)

    def run(self):
        self.root = Tk()
        self.root.title("Arb Finder")
        self.root.geometry("300x100")

        app = Frame(self.root)
        app.grid()

        start = Button(app, text="Start", command=self.start)
        stop = Button(app, text="Stop", command=self.stop)
        start.grid(row=0, column=0, padx=(40, 40), pady=(40, 40))
        stop.grid(row=0, column=1, padx=(40, 40), pady=(40, 40))

        self.root.after(1, self.trading)
        self.root.mainloop()

app = App()
app.run()
